# NuclearPy - Pseudotime analysis

This interactive notebook describes the workflow to analyze previously-processed NuclearPy data for pseudotime analysis.


## Pre-requisite

Please make sure that `NuclearPy` is successfully installed on your local computer. Installation instructions can be found on `NuclearPy` [GitHub repository](https://github.com/f-hamidlab/nuclearpy).


We begin by importing the Analyzor class from `nuclearpy` module.

In [ ]:
from nuclearpy.analyzer import Analyzor
import matplotlib.pyplot as plt
import scanpy as sc

Pass the path to example output to `Analyzor` class function:

In [ ]:
obj = Analyzor(collated_csv = "../data/sample_output/Ngn2_Temporal.csv")

Normalize DAPI channell and keep predicted single cells.

In [ ]:
obj.normChannel(channel = "dapi", method = "mode", nbins = 40, intensity_type = "total")
obj.findSingleCells(byExperiment = True, nbins = 40, spread = 0.4, channel = "dapi")
obj.filterCells(filter = "isSingleCell == True", inplace = True)

Next, choose the variable features to be included in the pseudotime analysis. <br>
These are the list of all varible feature:

In [ ]:
obj.showADataVars()

These are the features we include in the analysis:

- avg_intensity_dapi
- nuclear_area
- major_axis
- avg_intensity_internal_ring_dapi
- avg_intensity_beta3
- avg_intensity_rfp
- dna_dots
- spatial_entropy

In [ ]:
include_feat = ['avg_intensity_dapi',
 'nuclear_area',
 'major_axis',
 'avg_intensity_internal_ring_dapi',
 'avg_intensity_beta3',
 'avg_intensity_rfp',
 'dna_dots',
 'spatial_entropy']
obj.includeVars(vars = include_feat)

Construct diffusion map

#### DIFFMAP

In [ ]:
obj.findNeighbours(method = "umap")
obj.findClusters(method = "leiden", res=.15)
obj.runDimReduc(method = "diffmap")

In [ ]:
obj.plotDim(hue = "leiden", method="diffmap")

Choose root cells for diffusion pseudotime

In [ ]:
%matplotlib tk
root_cells = obj.chooseCells(reduction = "diffmap")

In [ ]:
%matplotlib inline

In [ ]:
obj.runPT(root_cells = root_cells)
obj.plotDim(hue = "dpt_pseudotime", method="diffmap")

Visualize changes in features between clusters:

In [ ]:
obj.normAData(method = "standardscaler")
fig, ax = plt.subplots(figsize = (5, 7))

markers = ['nuclear_area','avg_intensity_dapi','major_axis','dna_dots','avg_intensity_internal_ring_dapi']
sc.pl.stacked_violin(obj.adata, markers, groupby = 'leiden', swap_axes = True, ax = ax, dendrogram = False, categories_order=['2','1','0'])
fig.tight_layout()
plt.show()

Visualize changes in features across pseudotime

In [ ]:
pseudotime_path = [2,1,0]
markers = ['nuclear_area',
           'avg_intensity_dapi',
           'nuclear_perimeter',
           'major_axis',
           'minor_axis',
           'axes_ratio',
           'circularity',
           'eccentricity',
           'solidity',
           'avg_intensity_core_dapi',
           'avg_intensity_internal_ring_dapi',
           'avg_intensity_external_ring_dapi', 
           'avg_intensity_beta3',
           'avg_intensity_rfp',
          'dna_peaks',
          'dna_dots',
          'dna_dots_size_median']
sc.pl.paga_path(
    obj.adata, 
    pseudotime_path, 
    markers,
    show_node_names = True,
    n_avg = 50,
    annotations = ['dpt_pseudotime'],
    show_colorbar = True,
    color_map = 'coolwarm',
    groups_key = 'leiden',
    color_maps_annotations = {'dpt_pseudotime': 'viridis'},
    title = 'Path',
    return_data = False,
    normalize_to_zero_one = True,
    show = True
)

In [ ]:
pseudotime_path = [0,1,2,3,4,5]
markers = ['nuclear_area',
           'avg_intensity_dapi',
           'nuclear_perimeter',
           'major_axis',
           'minor_axis',
           'axes_ratio',
           'circularity',
           'eccentricity',
           'solidity',
           'avg_intensity_core_dapi',
           'avg_intensity_internal_ring_dapi',
           'avg_intensity_external_ring_dapi', 
           'avg_intensity_beta3',
           'avg_intensity_rfp',
          'dna_peaks',
          'dna_dots',
          'dna_dots_size_median']
sc.pl.paga_path(
    obj.adata, 
    pseudotime_path, 
    markers,
    show_node_names = True,
    n_avg = 50,
    annotations = ['dpt_pseudotime'],
    show_colorbar = True,
    color_map = 'coolwarm',
    groups_key = 'experiment',
    color_maps_annotations = {'dpt_pseudotime': 'viridis'},
    title = 'Path',
    return_data = False,
    normalize_to_zero_one = True,
    show = True
)